# Initialization


In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn import ensemble
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score, cross_val_predict, train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
import snips as snp


Reading part (1/n) of the data

In [2]:
def skiprow(i, n):
    if i % n == 0:
       return False
    return True


In [3]:
n=5
train = pd.read_csv('../data/mnist-in-csv/mnist_train.csv.zip', compression='zip'
                    , skiprows = lambda x: skiprow(x, n))
test = pd.read_csv('../data/mnist-in-csv/mnist_test.csv.zip', compression='zip'
                  , skiprows = lambda x: skiprow(x, 1))

train.shape

(12000, 785)

In [4]:
train.sample(5)

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
3911,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3136,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9400,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4026,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7436,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
df=train.drop('label', axis = 1)
df.sample(5)

,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,1x10,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
5882,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7355,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9367,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8578,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8507,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Feature selection

In [ ]:
y=train.label
X=train.drop('label', axis=1)
X_test = test.drop('label', axis=1)


# Define models

Multi label models.

In [ ]:
models = []
models.append(('KNN', KNeighborsClassifier(3)))
models.append(('Linear SVM', SVC(gamma='scale', C=0.025, decision_function_shape='ovo')))
models.append(('Neural Net', MLPClassifier(alpha=1, solver='sgd')))
models.append(('XGB', XGBClassifier(verbosity=2, objective='multi:softmax', num_class=10)))
models.append(('LR', LogisticRegression(solver="liblinear", multi_class="multinomial", max_iter=1000))) # ovr - one versus reset


In [ ]:
#models.append(('RBF SVM', SVC(gamma=2, C=1)))
#models.append(('Gaussian', GaussianProcessClassifier(1.0 * RBF(1.0))))
#models.append(('DT', DecisionTreeClassifier(max_depth=5)))
#models.append(('RF', RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1)))
#models.append(('Ada', AdaBoostClassifier()))
#models.append(('NB', GaussianNB()))
#models.append(('QDA', QuadraticDiscriminantAnalysis()))

# Model fitting and prediction

In [ ]:
def model_score(name, model, X, y, val_y, y_pred, y_test, icv):
    #print('Running CV score')
    cvscore = cross_val_score(model, X, y, cv=icv)
    print('Model {0} score: {1:.4f} and std dev: {2:.4f}'.format(name, cvscore.mean(), cvscore.std()))
    print('Validation variance score: %.4f' % r2_score(val_y, y_pred))
    print('Test variance score: %.4f' % r2_score(test.label, y_test))

    return cvscore, r2_score(val_y, y_pred), r2_score(test.label, y_test)

In [ ]:
def cv_fit_and_predict(models, X, y):
    results = []
    train_X, val_X, train_y, val_y = train_test_split(X, y, random_state = 0)
    for name, model in models:
        #print("Fitting model {}".format(name))
        m = model.fit(train_X, train_y)
        #print("Fitted model is {}".format(m))
        #print('Running prediction')
        y_pred = model.predict(val_X)
        y_test = model.predict(X_test)
        score, val_r2, test_r2 = model_score(name, model, X, y, val_y, y_pred, y_test, 10)
        results.append((name, score, val_r2, test_r2))
    print("Done")
    return results


In [ ]:
results = cv_fit_and_predict(models, X, y)

Model KNN score: 0.9483 and std dev: 0.0072
Validation variance score: 0.8706
Test variance score: 0.8793


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this 

# Validation and results

In [ ]:
def model_validation(results):
    fig, ax = plt.subplots(figsize=(8, 5))
    ax.boxplot(results[:,1])
    ax.set_xticklabels(results[:,0])
    ax.set_title('Model CV score')
    plt.show()
    
    plt.figure(figsize=(8, 5))
    plt.bar(results[:,0], results[:,2])
    plt.xlabel('Models')
    plt.ylabel('Var score')
    plt.title('Models variance score')
    plt.show()
    
    plt.figure(figsize=(8, 5))
    plt.bar(results[:,0], results[:,3])
    plt.xlabel('Models')
    plt.ylabel('Var score')
    plt.title('Test variance score')
    plt.show()
    
    return 0

In [ ]:
np_results=np.array(results)

In [ ]:
status = model_validation(np_results)